In [1]:
# !pip install skforecast

In [2]:
# !pip install scikit-learn-intelex -q --progress-bar off


# from sklearnex import patch_sklearn
# patch_sklearn()

In [3]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression, LinearRegression, HuberRegressor, Ridge, Lasso, SGDClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import RFE, RFECV, mutual_info_regression
from sklearn.metrics import accuracy_score, mean_absolute_error
# from skforecast.ForecasterAutoreg import ForecasterAutoreg
# from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
# from skforecast.ForecasterAutoregMultiOutput import ForecasterAutoregMultiOutput
# from skforecast.model_selection import grid_search_forecaster
# from skforecast.model_selection import backtesting_forecaster

from joblib import dump, load

# Dimensionality Reduction / Clustering
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


# Metrics
from sklearn.metrics import precision_recall_fscore_support, roc_curve, auc
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5

import warnings
warnings.filterwarnings('ignore')

print("Setup done")

Setup done


In [4]:
df_train = pd.read_csv('../input/tabular-playground-series-mar-2022/train.csv')
df_test = pd.read_csv('../input/tabular-playground-series-mar-2022/test.csv')

for df in [df_train,df_test]:
    df['time']=pd.to_datetime(df['time'],format='%Y-%m-%d %H:%M:%S')
#     df.set_index('time',inplace=True)
    df=df.sort_index()

In [5]:
df_train.head()

,row_id,time,x,y,direction,congestion
0,0,1991-04-01,0,0,EB,70
1,1,1991-04-01,0,0,NB,49
2,2,1991-04-01,0,0,SB,24
3,3,1991-04-01,0,1,EB,18
4,4,1991-04-01,0,1,NB,60


In [6]:
df_test.head()

,row_id,time,x,y,direction
0,848835,1991-09-30 12:00:00,0,0,EB
1,848836,1991-09-30 12:00:00,0,0,NB
2,848837,1991-09-30 12:00:00,0,0,SB
3,848838,1991-09-30 12:00:00,0,1,EB
4,848839,1991-09-30 12:00:00,0,1,NB


In [7]:
df_test.head()

,row_id,time,x,y,direction
0,848835,1991-09-30 12:00:00,0,0,EB
1,848836,1991-09-30 12:00:00,0,0,NB
2,848837,1991-09-30 12:00:00,0,0,SB
3,848838,1991-09-30 12:00:00,0,1,EB
4,848839,1991-09-30 12:00:00,0,1,NB


## Feature creation

In [8]:
memorial_day=['1991-05-27']
independence_day=['1991-07-04']
labor_day=['1991-09-02']
holidays=[memorial_day,independence_day,labor_day]

In [9]:
for df in [df_train,df_test]:
    df['hour']=df['time'].dt.time
    df['day_of_week']=df['time'].dt.dayofweek
    df['name_day_week']=df['time'].dt.day_name()
    df['day_of_month']=df['time'].dt.day
    df['relative_day']=df['time'].dt.dayofyear-90
    df['relative_month']=df['time'].dt.month-3
    df['relative_week']=df['time'].dt.isocalendar().week-13
    df['node']=df_train['x'].astype('str')+df_train['y'].astype('str')
    df['road']=df['x'].astype('str')+df['y'].astype('str')+df['direction'].astype('str')
    #df['holiday']=((df.time.dt.date.astype('str').isin(holidays))).astype('int')
    df['memorial_day']=((df.time.dt.date.astype('str').isin(memorial_day))).astype('int')
    df['independence_day']=((df.time.dt.date.astype('str').isin(independence_day))).astype('int')
    df['labor_day']=((df.time.dt.date.astype('str').isin(labor_day))).astype('int')
    

# df_train=pd.concat([df_train,pd.get_dummies(df_train['road'])],axis=1)
# df_test=pd.concat([df_test,pd.get_dummies(df_test['road'])],axis=1)

In [10]:
road_encoding = {k: i for i, k in enumerate(df['road'].unique(), 0) }
        
df_train['road_enc']=df_train['road'].map(road_encoding)
df_test['road_enc']=df_test['road'].map(road_encoding)

In [11]:
le=LabelEncoder()
df_train['hour_enc']=le.fit_transform(df_train['hour'])
df_test['hour_enc']=le.transform(df_test['hour'])

In [12]:
le2=LabelEncoder()
df_train['direction_enc']=le2.fit_transform(df_train['direction'])
df_test['direction_enc']=le2.transform(df_test['direction'])

In [13]:
df_train.head()

,row_id,time,x,y,direction,congestion,hour,day_of_week,name_day_week,day_of_month,...,relative_month,relative_week,node,road,memorial_day,independence_day,labor_day,road_enc,hour_enc,direction_enc
0,0,1991-04-01,0,0,EB,70,00:00:00,0,Monday,1,...,1,1,00,00EB,0,0,0,0,0,0
1,1,1991-04-01,0,0,NB,49,00:00:00,0,Monday,1,...,1,1,00,00NB,0,0,0,1,0,1
2,2,1991-04-01,0,0,SB,24,00:00:00,0,Monday,1,...,1,1,00,00SB,0,0,0,2,0,4
3,3,1991-04-01,0,1,EB,18,00:00:00,0,Monday,1,...,1,1,01,01EB,0,0,0,3,0,0
4,4,1991-04-01,0,1,NB,60,00:00:00,0,Monday,1,...,1,1,01,01NB,0,0,0,4,0,1


In [14]:
df_test.head()

,row_id,time,x,y,direction,hour,day_of_week,name_day_week,day_of_month,relative_day,relative_month,relative_week,node,road,memorial_day,independence_day,labor_day,road_enc,hour_enc,direction_enc
0,848835,1991-09-30 12:00:00,0,0,EB,12:00:00,0,Monday,30,183,6,27,00,00EB,0,0,0,0,36,0
1,848836,1991-09-30 12:00:00,0,0,NB,12:00:00,0,Monday,30,183,6,27,00,00NB,0,0,0,1,36,1
2,848837,1991-09-30 12:00:00,0,0,SB,12:00:00,0,Monday,30,183,6,27,00,00SB,0,0,0,2,36,4
3,848838,1991-09-30 12:00:00,0,1,EB,12:00:00,0,Monday,30,183,6,27,01,01EB,0,0,0,3,36,0
4,848839,1991-09-30 12:00:00,0,1,NB,12:00:00,0,Monday,30,183,6,27,01,01NB,0,0,0,4,36,1


## Feature Engineering

In [15]:
12*3*65
#12 hours * 3 measures/hour * 65 roads

2340

In [16]:
df_train.columns.to_list()

['row_id',
 'time',
 'x',
 'y',
 'direction',
 'congestion',
 'hour',
 'day_of_week',
 'name_day_week',
 'day_of_month',
 'relative_day',
 'relative_month',
 'relative_week',
 'node',
 'road',
 'memorial_day',
 'independence_day',
 'labor_day',
 'road_enc',
 'hour_enc',
 'direction_enc']

In [17]:
features = [
# 'row_id',
# 'time',
# 'x',
# 'y',
# 'direction',
# 'congestion',
# 'hour',
'day_of_week',
# 'day_of_month',
# 'relative_day',
# 'relative_month',
# 'relative_week',
# 'road',
'memorial_day',
'independence_day',
'labor_day',
'road_enc',
'hour_enc',
#'direction_enc'
]

In [18]:
end_train = '1991-08-24 12:00:00'
end_validation = '1991-09-30 11:40:00'
X_train = df_train.loc[df_train.time<end_train, features+['time']].set_index('time')
y_train = df_train.loc[df_train.time<end_train, ['congestion','time']].set_index('time')

X_valid = df_train.loc[df_train.time>=end_train, features+['time']].set_index('time')
y_valid = df_train.loc[df_train.time>=end_train, ['congestion','time']].set_index('time')

X_test = df_test.loc[:,features+['time']].set_index('time')

print(f"Dates train      : {X_train.index.min()} --- {X_train.index.max()}  (n={len(X_train)})")
print(f"Dates validacion : {X_valid.index.min()} --- {X_valid.index.max()}  (n={len(X_valid)})")
print(f"Dates test       : {X_test.index.min()} --- {X_test.index.max()}  (n={len(X_test)})")

Dates train      : 1991-04-01 00:00:00 --- 1991-08-24 11:40:00  (n=678925)
Dates validacion : 1991-08-24 12:00:00 --- 1991-09-30 11:40:00  (n=169910)
Dates test       : 1991-09-30 12:00:00 --- 1991-09-30 23:40:00  (n=2340)


In [19]:
# end_train = '1991-08-24 12:00:00'
# end_validation = '1991-09-30 11:40:00'
# X_train = df_train.loc[df_train.time<end_train, features+['time']].set_index('time')
# y_train = df_train.loc[df_train.time<end_train, ['congestion','time']].set_index('time')
# X_valid = df_train.loc[df_train.time>=end_train, features+['time']].set_index('time')
# y_valid = df_train.loc[df_train.time>=end_train, ['congestion','time']].set_index('time')

# X_test = df_test.loc[:,features+['time']].set_index('time')

# print(f"Dates train      : {X_train.index.min()} --- {X_train.index.max()}  (n={len(X_train)})")
# print(f"Dates validacion : {X_valid.index.min()} --- {X_valid.index.max()}  (n={len(X_valid)})")
# print(f"Dates test       : {X_test.index.min()} --- {X_test.index.max()}  (n={len(X_test)})")


# regressor = RandomForestRegressor(n_estimators=100, random_state=0)
# # regressor = DecisionTreeRegressor(random_state=0)

# regressor.fit(X_train, y_train)
# print("Valid-set R^2: {:.2f}".format(regressor.score(X_valid, y_valid)))

# y_pred_valid = regressor.predict(X_valid)
# print("Valid-set MAE: {:.2f}".format(mean_absolute_error(y_valid,y_pred_valid)))

# y_pred_train = regressor.predict(X_train)
# df_pred_train=pd.DataFrame({'time':X_train.index,'predictions':regressor.predict(X_train)})
# df_pred_valid=pd.DataFrame({'time':X_valid.index,'predictions':regressor.predict(X_valid)})

In [20]:
# df_pred_train=pd.DataFrame({'time':X_train.index,'predictions':regressor.predict(X_train)})
# df_pred_valid=pd.DataFrame({'time':X_valid.index,'predictions':regressor.predict(X_valid)})
# df_pred_train.groupby('time')['predictions'].mean().values

In [21]:
# plt.figure()

# nb_readings_day=24*3 #number of readings per day for each of the 65 roads #24 hours * 3 readings/hour
# nb_train_days_to_show=nb_readings_day*14
# nb_valid_days_to_show=nb_readings_day*14

# n_train=len(X_train.index.unique()) #number of different times in train. Ex of time format: 1991-04-01 00:00:00
# n_valid=len(X_valid.index.unique()) #number of different times in valid  Ex of time format: 1991-08-24 12:00:00

# y_train_mean=y_train.reset_index().groupby('time').mean()
# y_valid_mean=y_valid.reset_index().groupby('time').mean()

# y_train_pred_mean=df_pred_train.groupby('time')['predictions'].mean().values
# y_valid_pred_mean=df_pred_valid.groupby('time')['predictions'].mean().values

# plt.figure(figsize=(14, 10))
# xticks = pd.date_range(start=df_train.time.min(), end=df_train.time.max(),freq='D')[-nb_train_days_to_show:]

# plt.xticks(range(0, n_train+n_valid+nb_readings_day,nb_readings_day)[-nb_train_days_to_show:], xticks.strftime("%a %m-%d"), rotation=90,ha="left")
# plt.plot(range(n_train)[-nb_train_days_to_show:], y_train_mean[-nb_train_days_to_show:], label="train")
# plt.plot(range(n_train, n_train+n_valid)[:nb_valid_days_to_show], y_valid_mean[:nb_valid_days_to_show], '-', label="valid")
# plt.plot(range(n_train)[-nb_train_days_to_show:], y_train_pred_mean[-nb_train_days_to_show:], '--', color='lightgreen',label="prediction train",alpha=1)
# plt.plot(range(n_train, n_train+n_valid)[:nb_valid_days_to_show], y_valid_pred_mean[:nb_valid_days_to_show],'--',color='lightgreen',label="prediction valid",alpha=1)
# plt.legend(loc=(1.01, 0))
# plt.xlabel("Date")
# plt.ylabel("Congestion")

In [22]:
end_train = '1991-08-24 12:00:00'
end_validation = '1991-09-30 11:40:00'
# function to evaluate and plot a regressor on a given feature set

def eval_on_features(train, features, regressor):
# split the given features into a training and a test set
    
    X_train = train.loc[train.time<end_train, features].set_index('time')
    y_train = train.loc[train.time<end_train, ['congestion','time']].set_index('time')
    X_valid = train.loc[train.time>=end_train, features].set_index('time')
    y_valid = train.loc[train.time>=end_train, ['congestion','time']].set_index('time')

#     X_test = df_test.loc[:,features+['time']].set_index('time')

    print(f"Dates train      : {X_train.index.min()} --- {X_train.index.max()}  (n={len(X_train)})")
    print(f"Dates validacion : {X_valid.index.min()} --- {X_valid.index.max()}  (n={len(X_valid)}) \n")
#     print(f"Dates test       : {X_test.index.min()} --- {X_test.index.max()}  (n={len(X_test)})")


    regressor = RandomForestRegressor(n_estimators=100, random_state=0)
    # regressor = DecisionTreeRegressor(random_state=0)

    regressor.fit(X_train, y_train)
    print("Valid-set R^2: {:.2f}".format(regressor.score(X_valid, y_valid)))

    y_pred_valid = regressor.predict(X_valid)
    print("Valid-set MAE: {:.2f}".format(mean_absolute_error(y_valid,y_pred_valid)))

    y_pred_train = regressor.predict(X_train)
    df_pred_train=pd.DataFrame({'time':X_train.index,'predictions':regressor.predict(X_train)})
    df_pred_valid=pd.DataFrame({'time':X_valid.index,'predictions':regressor.predict(X_valid)})

    plt.figure()

    nb_readings_day=24*3 #number of readings per day for each of the 65 roads #24 hours * 3 readings/hour
    nb_train_days_to_show=nb_readings_day*14
    nb_valid_days_to_show=nb_readings_day*14

    n_train=len(X_train.index.unique()) #number of different times in train. Ex of time format: 1991-04-01 00:00:00
    n_valid=len(X_valid.index.unique()) #number of different times in valid  Ex of time format: 1991-08-24 12:00:00

    y_train_mean=y_train.reset_index().groupby('time').mean()
    y_valid_mean=y_valid.reset_index().groupby('time').mean()

    y_train_pred_mean=df_pred_train.groupby('time')['predictions'].mean().values
    y_valid_pred_mean=df_pred_valid.groupby('time')['predictions'].mean().values

    plt.figure(figsize=(14, 10))
    xticks = pd.date_range(start=df_train.time.min(), end=df_train.time.max(),freq='D')[-nb_train_days_to_show:]

    plt.xticks(range(0, n_train+n_valid+nb_readings_day,nb_readings_day)[-nb_train_days_to_show:], xticks.strftime("%a %m-%d"), rotation=90,ha="left")
    plt.plot(range(n_train)[-nb_train_days_to_show:], y_train_mean[-nb_train_days_to_show:], label="train")
    plt.plot(range(n_train, n_train+n_valid)[:nb_valid_days_to_show], y_valid_mean[:nb_valid_days_to_show], '-', label="valid")
    plt.plot(range(n_train)[-nb_train_days_to_show:], y_train_pred_mean[-nb_train_days_to_show:], '--', color='lightgreen',label="prediction train",alpha=1)
    plt.plot(range(n_train, n_train+n_valid)[:nb_valid_days_to_show], y_valid_pred_mean[:nb_valid_days_to_show],'--',color='lightgreen',label="prediction valid",alpha=1)
    plt.legend(loc=(1.01, 0))
    plt.xlabel("Date")
    plt.ylabel("Congestion")

In [23]:
features = [
# 'row_id',
'time',
#'x',
# 'y',
# 'direction',
# 'congestion',
# 'hour',
'day_of_week',
#'day_of_month',
#relative_day',
# 'relative_month',
# 'relative_week',
# 'road',
'memorial_day',
'independence_day',
'labor_day',
'road_enc',
'hour_enc',
#'direction_enc'
]

## Decision Tree

In [24]:
# from sklearn.tree import DecisionTreeRegressor
# regressor = DecisionTreeRegressor(random_state=0)
# plt.figure()
# eval_on_features(df_train, features,regressor)

In [25]:
# X_train = df_train.loc[df_train.time<end_train, features].set_index('time')
# y_train = df_train.loc[df_train.time<end_train, ['congestion','time']].set_index('time')
# X_valid = df_train.loc[df_train.time>=end_train, features].set_index('time')
# y_valid = df_train.loc[df_train.time>=end_train, ['congestion','time']].set_index('time')

In [26]:
# regressor.fit(X_train,y_train)
# pd.DataFrame(regressor.feature_importances_,index=X_train.columns)

## Random Forest tree

In [27]:
# regressor = RandomForestRegressor(n_estimators=100, random_state=0)
# plt.figure()
# eval_on_features(df_train, features,regressor) 

## SK Forecast

In [28]:
features = [
# 'row_id',
'time',
#'x',
# 'y',
# 'direction',
# 'congestion',
# 'hour',
'day_of_week',
#'day_of_month',
#relative_day',
# 'relative_month',
# 'relative_week',
# 'road',
'memorial_day',
'independence_day',
'labor_day',
'road_enc',
'hour_enc',
#'direction_enc'
]

In [29]:
df_test[features]

,time,day_of_week,memorial_day,independence_day,labor_day,road_enc,hour_enc
0,1991-09-30 12:00:00,0,0,0,0,0,36
1,1991-09-30 12:00:00,0,0,0,0,1,36
2,1991-09-30 12:00:00,0,0,0,0,2,36
3,1991-09-30 12:00:00,0,0,0,0,3,36
4,1991-09-30 12:00:00,0,0,0,0,4,36
...,...,...,...,...,...,...,...
2335,1991-09-30 23:40:00,0,0,0,0,60,71
2336,1991-09-30 23:40:00,0,0,0,0,61,71
2337,1991-09-30 23:40:00,0,0,0,0,62,71
2338,1991-09-30 23:40:00,0,0,0,0,63,71


In [30]:
X_train = df_train.loc[df_train.time<end_train, features+['row_id']].set_index('time')
y_train = df_train.loc[df_train.time<end_train, ['congestion','time']].set_index('time')
X_valid = df_train.loc[df_train.time>=end_train, features+['row_id']].set_index('time')
y_valid = df_train.loc[df_train.time>=end_train, ['congestion','time']].set_index('time')
X_test = df_test[features]

In [31]:
X_train_full=pd.concat([X_train,X_valid],axis=0)
y_train_full=pd.concat([y_train,y_valid],axis=0)

In [32]:
features = [
# 'row_id',
#'time',
#'x',
# 'y',
# 'direction',
# 'congestion',
# 'hour',
'day_of_week',
#'day_of_month',
#relative_day',
# 'relative_month',
# 'relative_week',
# 'road',
'memorial_day',
'independence_day',
'labor_day',
'road_enc',
'hour_enc',
#'direction_enc'
]

In [33]:
regressor = DecisionTreeRegressor(random_state=0)
#regressor = DecisionTreeRegressor(random_state=0,criterion='absolute_error') #try absolute error
regressor.fit(X_train_full[features],y_train_full)

DecisionTreeRegressor(random_state=0)

In [34]:
predictions = pd.DataFrame({'row_id':df_test['row_id'],'congestion':regressor.predict(X_test[features])})

In [35]:
predictions

,row_id,congestion
0,848835,46.250000
1,848836,35.333333
2,848837,57.125000
3,848838,22.875000
4,848839,73.291667
...,...,...
2335,851170,68.000000
2336,851171,34.291667
2337,851172,70.958333
2338,851173,17.291667


## Submission

In [36]:
predictions.to_csv('baseline_2.csv',index=False)

In [37]:
# !cat submission_last_day.csv